In [15]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
from scipy.linalg import sqrtm
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Function to load images
def load_images_from_files(file_list, img_size=(32, 32)):
    images = []
    for file_path in file_list:
        img = Image.open(file_path).convert('RGB')  # Convert to RGB
        img = img.resize(img_size)  # Resize to required dimensions
        img = np.asarray(img) / 255.0  # Normalize to [0, 1]
        images.append(img)
    return np.array(images)

# List of your uploaded image files
file_list = [
    '/content/generated_plot_e010.png',   # Update with actual file names
    '/content/generated_plot_e020.png',
    '/content/generated_plot_e030.png',
    '/content/generated_plot_e040.png',
    '/content/generated_plot_e050.png',
    '/content/generated_plot_e060.png',
    '/content/generated_plot_e070.png',
    '/content/generated_plot_e080.png',
    '/content/generated_plot_e090.png',
    '/content/generated_plot_e100.png'
]

# Load the images
fake_images = load_images_from_files(file_list)

# Convert the list of images to a numpy array
fake_images = np.array(fake_images)  # Shape: (N, 32, 32, 3)

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to match the generated images
    transforms.ToTensor()          # Convert images to tensors
])

cifar10_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Get a subset of real images (let's say 100 for the evaluation)
real_images = []
for i in range(100):  # Load 100 real images
    img, _ = cifar10_dataset[i]  # Get image and label (label is not needed here)
    real_images.append(img.numpy().transpose(1, 2, 0))  # Convert to numpy array

real_images = np.array(real_images)  # Convert list to numpy array

# Function to calculate Inception Score (IS)
def calculate_inception_score(images, splits=10):
    inception_model = inception_v3(pretrained=True, transform_input=False).eval().cuda()
    preds = []

    for img in images:
        img = torch.from_numpy(img).float().permute(2, 0, 1).unsqueeze(0).cuda()
        img = F.interpolate(img, size=(299, 299), mode='bilinear', align_corners=False)
        with torch.no_grad():
            pred = F.softmax(inception_model(img), dim=1).cpu().data.numpy()
        preds.append(pred)

    preds = np.concatenate(preds, axis=0)
    split_scores = []

    for i in range(splits):
        part = preds[i * (len(preds) // splits): (i + 1) * (len(preds) // splits), :]
        kl_divergence = part * (np.log(part + 1e-10) - np.log(np.expand_dims(np.mean(part, 0) + 1e-10, 0)))
        kl_divergence = np.sum(kl_divergence, axis=1)
        split_scores.append(np.exp(np.mean(kl_divergence)))

    return np.mean(split_scores), np.std(split_scores)

# Function to calculate Fréchet Inception Distance (FID)
def calculate_fid(real_images, fake_images):
    inception_model = inception_v3(pretrained=True, transform_input=False).eval().cuda()

    def get_inception_features(images):
        features = []
        for img in images:
            img = img.reshape(32, 32, 3)  # Ensure img is a 3D array (height, width, channels)
            img = torch.from_numpy(img).float().permute(2, 0, 1).unsqueeze(0).cuda()  # Shape: (1, 3, 32, 32)
            img = F.interpolate(img, size=(299, 299), mode='bilinear', align_corners=False)  # Resize to Inception input size
            with torch.no_grad():
                feature = inception_model(img)  # Get the feature representation
                features.append(feature.cpu().numpy().reshape(-1))
        return np.array(features)

    # Extract features for real and fake images
    real_features = get_inception_features(real_images)
    fake_features = get_inception_features(fake_images)

    # Calculate mean and covariance for real and fake features
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_fake, sigma_fake = np.mean(fake_features, axis=0), np.cov(fake_features, rowvar=False)

    # Calculate squared difference between means
    mu_diff = mu_real - mu_fake

    # Calculate the sqrt of the product of covariances
    covmean = sqrtm(sigma_real.dot(sigma_fake))

    # Check and correct imaginary component
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Calculate FID
    fid = np.sum(mu_diff ** 2) + np.trace(sigma_real + sigma_fake - 2 * covmean)
    return fid

# Calculate Inception Score (IS) for fake images
is_mean, is_std = calculate_inception_score(fake_images)
print(f'Inception Score: mean={is_mean:.2f}, std={is_std:.2f}')

# Calculate Fréchet Inception Distance (FID)
fid_value = calculate_fid(real_images, fake_images)
print(f'FID: {fid_value:.2f}')


Files already downloaded and verified
Inception Score: mean=1.00, std=0.00
FID: 2250.34
